<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/DL_NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Making sure you have all the necessary libraries installed.

In [27]:
#!pip install transformers datasets evaluate
#! pip install huggingface_hub
#!pip install datasets
#! pip install transformers

 login to your Hugging Face account

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Loading SQuAD dataset from Datasets Library.

In [8]:
from datasets import load_dataset

#Loading smaller subets of data to make sure that wholepipe line works before training with whole data.
squad = load_dataset("squad", split="train[:5000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Spliting the dataset’s train split into a train and test set with the train_test_split method:

In [9]:
squad = squad.train_test_split(test_size=0.2)

Taking a look at the example.

In [37]:
squad["train"][0]

{'id': '5734296dd058e614000b6a71',
 'title': 'Montana',
 'context': 'When the U.S. entered World War II on December 8, 1941, many Montanans already had enlisted in the military to escape the poor national economy of the previous decade. Another 40,000-plus Montanans entered the armed forces in the first year following the declaration of war, and over 57,000 joined up before the war ended. These numbers constituted about 10 percent of the state\'s total population, and Montana again contributed one of the highest numbers of soldiers per capita of any state. Many Native Americans were among those who served, including soldiers from the Crow Nation who became Code Talkers. At least 1500 Montanans died in the war. Montana also was the training ground for the First Special Service Force or "Devil\'s Brigade," a joint U.S-Canadian commando-style force that trained at Fort William Henry Harrison for experience in mountainous and winter conditions before deployment. Air bases were built in Gre

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [31]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [32]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [36]:
tokenized_squad["train"][0]

{'input_ids': [101,
  2040,
  4738,
  2012,
  1996,
  2510,
  5286,
  1999,
  8124,
  1029,
  102,
  2043,
  1996,
  1057,
  1012,
  1055,
  1012,
  3133,
  2088,
  2162,
  2462,
  2006,
  2285,
  1022,
  1010,
  3874,
  1010,
  2116,
  8124,
  3619,
  2525,
  2018,
  9417,
  1999,
  1996,
  2510,
  2000,
  4019,
  1996,
  3532,
  2120,
  4610,
  1997,
  1996,
  3025,
  5476,
  1012,
  2178,
  2871,
  1010,
  2199,
  1011,
  4606,
  8124,
  3619,
  3133,
  1996,
  4273,
  2749,
  1999,
  1996,
  2034,
  2095,
  2206,
  1996,
  8170,
  1997,
  2162,
  1010,
  1998,
  2058,
  5401,
  1010,
  2199,
  2587,
  2039,
  2077,
  1996,
  2162,
  3092,
  1012,
  2122,
  3616,
  11846,
  2055,
  2184,
  3867,
  1997,
  1996,
  2110,
  1005,
  1055,
  2561,
  2313,
  1010,
  1998,
  8124,
  2153,
  5201,
  2028,
  1997,
  1996,
  3284,
  3616,
  1997,
  3548,
  2566,
  8353,
  1997,
  2151,
  2110,
  1012,
  2116,
  3128,
  4841,
  2020,
  2426,
  2216,
  2040,
  2366,
  1010,
  2164,
  3548,
  20